## Inserting data in MySQL using Python

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
import requests

from datetime import date, datetime, timedelta

In [2]:
# Let's get the data from the Citibike API
url = 'http://www.citibikenyc.com/stations/json'
results = requests.get(url).json() 

In [3]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only we need
data = results["stationBeanList"]

In [4]:
len(data)

935

Now we will connect to our MySQL server. We will use the MySQLdb library of Python.

If you do not have the library, you need to install it by typing in the shell:


In [5]:
!sudo apt-get install python3-mysqldb
!sudo pip3 install -U sqlalchemy sql_magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-mysqldb is already the newest version (1.3.10-1build1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Requirement already up-to-date: sqlalchemy in /usr/local/lib/python3.6/dist-packages (1.3.14)
Requirement already up-to-date: sql_magic in /usr/local/lib/python3.6/dist-packages (0.0.4)


In [6]:
import sqlalchemy
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015')

engine = create_engine(conn_string)

Once we have connected successfully, we need to create our database:

In [7]:
# Query to create a database
# In this example, we will try to create the (existing) database "public"
# But in general, we can give any name to the database
db_name = 'public' 
create_db_query = f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"

# Create a database
engine.execute(create_db_query)

Then we create the table where we will store our data. For our example, we will just import three fields in the database: station_id, station_name, and number_of_docks

In [8]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid
if 'suffix' not in globals():
    suffix = str(uuid.uuid4())[:8]
print(suffix)

34064c0f


In [9]:
table_name = f'Docks_{suffix}'
# Create a table
create_table_query = f'''CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (station_id int, 
                                station_name varchar(250), 
                                number_of_docks int,
                                available_docks int,
                                date datetime,
                                PRIMARY KEY(station_id, date)
                                )'''
engine.execute(create_table_query)


Finally, we import the data into our table, using the INSERT command. 

In [10]:
query_template = f'''INSERT INTO {db_name}.{table_name}(station_id, 
                                            station_name, 
                                            number_of_docks, 
                                            available_docks, 
                                            date) 
                    VALUES (%s, %s, %s, %s, %s)'''

# THIS IS PROHIBITED
# query = "INSERT INTO citibike.Docks(station_id, station_name, number_of_docks) VALUES ("+entry["id"]+", "+entry["stationName"]+", "+entry["totalDocks"]+")"


for entry in data:
    dockid = entry["id"]
    addr = entry["stationName"]
    docks = entry["totalDocks"]
    available = entry["availableDocks"]
    # date =  datetime.now()
    # lastcommunicationtime is a string of 
    # the form "2016-02-09 10:16:49 AM"
    # See https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
    # to see the documentation on how to parse 
    date = datetime.strptime(entry["lastCommunicationTime"], '%Y-%m-%d %I:%M:%S %p')
    print("Inserting station", dockid, "at", addr)
    query_parameters = (dockid, addr, docks, available, date)
    engine.execute(query_template, query_parameters)



Inserting station 304 at Broadway & Battery Pl
Inserting station 359 at E 47 St & Park Ave
Inserting station 367 at E 53 St & Lexington Ave
Inserting station 402 at Broadway & E 22 St
Inserting station 3443 at W 52 St & 6 Ave
Inserting station 72 at W 52 St & 11 Ave
Inserting station 79 at Franklin St & W Broadway
Inserting station 82 at St James Pl & Pearl St
Inserting station 83 at Atlantic Ave & Fort Greene Pl
Inserting station 116 at W 17 St & 8 Ave
Inserting station 119 at Park Ave & St Edwards St
Inserting station 120 at Lexington Ave & Classon Ave
Inserting station 127 at Barrow St & Hudson St
Inserting station 128 at MacDougal St & Prince St
Inserting station 143 at Clinton St & Joralemon St
Inserting station 144 at Nassau St & Navy St
Inserting station 146 at Hudson St & Reade St
Inserting station 150 at E 2 St & Avenue C
Inserting station 151 at Cleveland Pl & Spring St
Inserting station 157 at Henry St & Atlantic Ave
Inserting station 161 at LaGuardia Pl & W 3 St
Inserting s

Inserting station 439 at E 4 St & 2 Ave
Inserting station 440 at E 45 St & 3 Ave
Inserting station 442 at W 27 St & 7 Ave
Inserting station 443 at Bedford Ave & S 9 St
Inserting station 445 at E 10 St & Avenue A
Inserting station 446 at W 24 St & 7 Ave
Inserting station 447 at 8 Ave & W 52 St
Inserting station 448 at W 37 St & 10 Ave
Inserting station 450 at W 49 St & 8 Ave
Inserting station 453 at W 22 St & 8 Ave
Inserting station 454 at E 51 St & 1 Ave
Inserting station 455 at 1 Ave & E 44 St
Inserting station 456 at E 53 St & Madison Ave
Inserting station 458 at 11 Ave & W 27 St
Inserting station 459 at W 20 St & 11 Ave
Inserting station 460 at S 4 St & Wythe Ave
Inserting station 461 at E 20 St & 2 Ave
Inserting station 462 at W 22 St & 10 Ave
Inserting station 464 at E 56 St & 3 Ave
Inserting station 465 at Broadway & W 41 St
Inserting station 466 at W 25 St & 6 Ave
Inserting station 467 at Dean St & 4 Ave
Inserting station 468 at Broadway & W 56 St
Inserting station 469 at Broadw

Inserting station 3139 at E 72 St & Park Ave
Inserting station 3140 at 1 Ave & E 78 St
Inserting station 3141 at 1 Ave & E 68 St
Inserting station 3142 at 1 Ave & E 62 St
Inserting station 3143 at 5 Ave & E 78 St
Inserting station 3144 at E 81 St & Park Ave
Inserting station 3145 at E 84 St & Park Ave
Inserting station 3146 at E 81 St & 3 Ave
Inserting station 3147 at E 85 St & 3 Ave
Inserting station 3148 at E 84 St & 1 Ave
Inserting station 3150 at E 85 St & York Ave
Inserting station 3151 at E 81 St & York Ave
Inserting station 3152 at 3 Ave & E 71 St
Inserting station 3154 at E 77 St & 3 Ave
Inserting station 3156 at E 72 St & York Ave
Inserting station 3157 at East End Ave & E 86 St
Inserting station 3158 at W 63 St & Broadway
Inserting station 3159 at W 67 St & Broadway
Inserting station 3160 at Central Park West & W 76 St
Inserting station 3161 at W 76 St & Columbus Ave
Inserting station 3163 at Central Park West & W 68 St
Inserting station 3164 at Columbus Ave & W 72 St
Inserti

Inserting station 3377 at Carroll St & Bond St
Inserting station 3378 at E 76 St & Park Ave
Inserting station 3379 at E 103 St & Lexington Ave
Inserting station 3381 at 3 St & Hoyt St
Inserting station 3382 at Carroll St & Smith St
Inserting station 3383 at Cathedral Pkwy & Broadway
Inserting station 3384 at Smith St & 3 St
Inserting station 3386 at 1 Pl & Clinton St
Inserting station 3387 at E 106 St & Madison Ave
Inserting station 3388 at President St & Henry St
Inserting station 3389 at Carroll St & Columbia St
Inserting station 3390 at E 109 St & 3 Ave
Inserting station 3391 at E 106 St & 1 Ave
Inserting station 3392 at Commerce St & Van Brunt St
Inserting station 3393 at Richards St & Delavan St
Inserting station 3394 at Columbia St & W 9 St
Inserting station 3395 at Henry St & W 9 St
Inserting station 3396 at Clinton St & 4 Place
Inserting station 3397 at Court St & Nelson St
Inserting station 3398 at Smith St & 9 St
Inserting station 3399 at 7 St & 3 Ave
Inserting station 3400 a

Inserting station 3606 at 49 Ave & 21 St
Inserting station 3607 at 31 Ave & 14 St
Inserting station 3608 at 5 St & 51 Ave
Inserting station 3609 at Vernon Blvd & 31 Ave
Inserting station 3610 at Vernon Blvd & 30 Rd
Inserting station 3611 at Vernon Blvd & 47 Rd
Inserting station 3612 at 30 Ave & 21 St
Inserting station 3613 at Center Blvd & 48 Ave
Inserting station 3614 at Crescent St & 30 Ave
Inserting station 3615 at 44 Dr & 21 St
Inserting station 3616 at Steinway St & 28 Ave
Inserting station 3617 at 28 Ave & 35 St
Inserting station 3618 at 27 St & Hunter St
Inserting station 3619 at Newtown Ave & 23 St
Inserting station 3620 at 11 St & 43 Ave
Inserting station 3621 at 27 Ave & 9 St
Inserting station 3622 at E 128 St & Madison Ave
Inserting station 3664 at North Moore St & Greenwich St
Inserting station 3623 at W 120 St & Claremont Ave
Inserting station 3628 at Lenox Ave & W 117 St
Inserting station 3629 at Adam Clayton Powell Blvd & W 126 St
Inserting station 3630 at Frederick Doug

Inserting station 3857 at Engert Ave & McGuinness Blvd
Inserting station 3858 at Kent St & McGuinness Blvd
Inserting station 3859 at Lorimer St & Calyer St
Inserting station 3860 at Wilson Ave & Troutman St
Inserting station 3861 at Menahan St & Wyckoff Ave
Inserting station 3862 at Linden St & Knickerbocker Ave
Inserting station 3863 at Central Ave & Woodbine St
Inserting station 3864 at Central Ave & Covert St
Inserting station 3865 at Knickerbocker Ave & Cooper St
Inserting station 3866 at Wilson Ave & Moffat St
Inserting station 3867 at Somers St & Broadway
Inserting station 3868 at Knickerbocker Ave & Halsey St
Inserting station 3869 at Van Buren St & Broadway
Inserting station 3870 at Melrose St & Broadway
Inserting station 3871 at Bushwick Ave & Furman Ave
Inserting station 3872 at Hancock St & Wyckoff Ave
Inserting station 3873 at Madison St & Cypress Ave
Inserting station 3874 at Menahan St & Seneca Ave
Inserting station 3875 at Stephen St & Seneca Ave
Inserting station 3876 a

Now let's see how to query the database

In [11]:
results = engine.execute(f"SELECT * FROM {db_name}.{table_name}")
rows = results.fetchall()
results.close()

In [12]:
for row in rows:
    print("Station ID:", row["station_id"])
    print("Station Name:", row["station_name"])
    print("Number of Docks:", row["number_of_docks"])
    print("Available Docks:", row["available_docks"])
    print("Last Communication:", row["date"])
    print("=============================================")

Station ID: 72
Station Name: W 52 St & 11 Ave
Number of Docks: 55
Available Docks: 49
Last Communication: 2020-03-11 11:28:15
Station ID: 79
Station Name: Franklin St & W Broadway
Number of Docks: 33
Available Docks: 0
Last Communication: 2020-03-11 11:28:16
Station ID: 82
Station Name: St James Pl & Pearl St
Number of Docks: 27
Available Docks: 0
Last Communication: 2020-03-11 11:30:38
Station ID: 83
Station Name: Atlantic Ave & Fort Greene Pl
Number of Docks: 62
Available Docks: 29
Last Communication: 2020-03-11 11:30:00
Station ID: 116
Station Name: W 17 St & 8 Ave
Number of Docks: 50
Available Docks: 1
Last Communication: 2020-03-11 11:30:22
Station ID: 119
Station Name: Park Ave & St Edwards St
Number of Docks: 19
Available Docks: 10
Last Communication: 2020-03-11 11:31:26
Station ID: 120
Station Name: Lexington Ave & Classon Ave
Number of Docks: 19
Available Docks: 17
Last Communication: 2020-03-11 11:31:16
Station ID: 127
Station Name: Barrow St & Hudson St
Number of Docks: 31
A

Station ID: 434
Station Name: 9 Ave & W 18 St
Number of Docks: 60
Available Docks: 2
Last Communication: 2020-03-11 11:31:25
Station ID: 435
Station Name: W 21 St & 6 Ave
Number of Docks: 50
Available Docks: 0
Last Communication: 2020-03-11 11:29:47
Station ID: 436
Station Name: Hancock St & Bedford Ave
Number of Docks: 27
Available Docks: 26
Last Communication: 2020-03-11 11:28:51
Station ID: 437
Station Name: Macon St & Nostrand Ave
Number of Docks: 24
Available Docks: 4
Last Communication: 2020-03-11 11:31:32
Station ID: 438
Station Name: St Marks Pl & 1 Ave
Number of Docks: 47
Available Docks: 46
Last Communication: 2020-03-11 11:30:08
Station ID: 439
Station Name: E 4 St & 2 Ave
Number of Docks: 39
Available Docks: 12
Last Communication: 2020-03-11 11:31:10
Station ID: 440
Station Name: E 45 St & 3 Ave
Number of Docks: 32
Available Docks: 0
Last Communication: 2020-03-11 11:28:26
Station ID: 442
Station Name: W 27 St & 7 Ave
Number of Docks: 51
Available Docks: 1
Last Communicatio

Available Docks: 2
Last Communication: 2020-03-11 11:30:47
Station ID: 3113
Station Name: Greenpoint Ave & Manhattan Ave
Number of Docks: 27
Available Docks: 24
Last Communication: 2020-03-11 11:28:39
Station ID: 3115
Station Name: India St & Manhattan Ave
Number of Docks: 50
Available Docks: 45
Last Communication: 2020-03-11 11:30:21
Station ID: 3116
Station Name: Huron St & Franklin St
Number of Docks: 35
Available Docks: 20
Last Communication: 2020-03-11 11:29:41
Station ID: 3117
Station Name: Franklin St & Dupont St
Number of Docks: 27
Available Docks: 22
Last Communication: 2020-03-11 11:28:07
Station ID: 3118
Station Name: McGuinness Blvd & Eagle St
Number of Docks: 42
Available Docks: 33
Last Communication: 2020-03-11 11:28:07
Station ID: 3119
Station Name: Vernon Blvd & 50 Ave
Number of Docks: 45
Available Docks: 6
Last Communication: 2020-03-11 11:27:58
Station ID: 3121
Station Name: Jackson Ave & 46 Rd
Number of Docks: 27
Available Docks: 1
Last Communication: 2020-03-11 11:2

Last Communication: 2020-03-11 11:30:45
Station ID: 3332
Station Name: Degraw St & Hoyt St
Number of Docks: 19
Available Docks: 9
Last Communication: 2020-03-11 11:31:03
Station ID: 3333
Station Name: Columbia St & Lorraine St
Number of Docks: 21
Available Docks: 1
Last Communication: 2020-03-11 11:30:55
Station ID: 3335
Station Name: Union St & 4 Ave
Number of Docks: 28
Available Docks: 3
Last Communication: 2020-03-11 11:28:57
Station ID: 3336
Station Name: E 97 St & Madison Ave
Number of Docks: 41
Available Docks: 36
Last Communication: 2020-03-11 11:30:42
Station ID: 3337
Station Name: Dwight St & Van Dyke St
Number of Docks: 23
Available Docks: 1
Last Communication: 2020-03-11 11:29:23
Station ID: 3338
Station Name: 2 Ave & E 99 St
Number of Docks: 31
Available Docks: 27
Last Communication: 2020-03-11 11:28:18
Station ID: 3339
Station Name: Berkeley Pl & 6 Ave
Number of Docks: 23
Available Docks: 20
Last Communication: 2020-03-11 11:27:50
Station ID: 3340
Station Name: Wolcott St 

Available Docks: 25
Last Communication: 2020-03-11 11:28:46
Station ID: 3551
Station Name: 3 Ave & E 112 St
Number of Docks: 23
Available Docks: 6
Last Communication: 2020-03-11 11:30:49
Station ID: 3552
Station Name: W 113 St & Broadway
Number of Docks: 39
Available Docks: 38
Last Communication: 2020-03-11 11:29:19
Station ID: 3553
Station Name: Frederick Douglass Blvd & W 112 St
Number of Docks: 37
Available Docks: 27
Last Communication: 2020-03-11 11:30:03
Station ID: 3554
Station Name: Vernon Blvd & 41 Rd
Number of Docks: 27
Available Docks: 7
Last Communication: 2020-03-11 11:27:41
Station ID: 3555
Station Name: 28 St & 41 Ave
Number of Docks: 27
Available Docks: 21
Last Communication: 2020-03-11 11:29:56
Station ID: 3557
Station Name: 40 Ave & 9 St
Number of Docks: 19
Available Docks: 0
Last Communication: 2020-03-11 11:28:25
Station ID: 3558
Station Name: Bergen St & Vanderbilt Ave
Number of Docks: 27
Available Docks: 25
Last Communication: 2020-03-11 11:30:38
Station ID: 3559
S

Available Docks: 2
Last Communication: 2020-03-11 11:31:19
Station ID: 3612
Station Name: 30 Ave & 21 St
Number of Docks: 15
Available Docks: 14
Last Communication: 2020-03-11 11:28:12
Station ID: 3613
Station Name: Center Blvd & 48 Ave
Number of Docks: 27
Available Docks: 7
Last Communication: 2020-03-11 11:30:29
Station ID: 3614
Station Name: Crescent St & 30 Ave
Number of Docks: 23
Available Docks: 23
Last Communication: 2020-03-11 11:31:04
Station ID: 3615
Station Name: 44 Dr & 21 St
Number of Docks: 21
Available Docks: 0
Last Communication: 2020-03-11 11:28:17
Station ID: 3616
Station Name: Steinway St & 28 Ave
Number of Docks: 25
Available Docks: 25
Last Communication: 2020-03-11 11:29:49
Station ID: 3617
Station Name: 28 Ave & 35 St
Number of Docks: 27
Available Docks: 26
Last Communication: 2020-03-11 11:28:53
Station ID: 3618
Station Name: 27 St & Hunter St
Number of Docks: 25
Available Docks: 13
Last Communication: 2020-03-11 11:31:30
Station ID: 3619
Station Name: Newtown Av

Number of Docks: 25
Available Docks: 11
Last Communication: 2020-03-11 09:09:03
Station ID: 3822
Station Name: Bushwick Ave & Forrest St
Number of Docks: 20
Available Docks: 19
Last Communication: 2020-03-11 11:28:37
Station ID: 3823
Station Name: Beaver St & Fayette St
Number of Docks: 30
Available Docks: 27
Last Communication: 2020-03-11 11:30:11
Station ID: 3824
Station Name: Van Sinderen Ave & Truxton St
Number of Docks: 23
Available Docks: 22
Last Communication: 2020-03-11 11:28:48
Station ID: 3825
Station Name: Broadway & Furman Ave
Number of Docks: 29
Available Docks: 29
Last Communication: 2020-03-11 11:27:42
Station ID: 3826
Station Name: Moffat St & Bushwick
Number of Docks: 18
Available Docks: 15
Last Communication: 2020-03-11 11:28:47
Station ID: 3827
Station Name: Halsey St & Broadway
Number of Docks: 20
Available Docks: 14
Last Communication: 2020-03-11 11:28:11
Station ID: 3828
Station Name: Eldert St & Bushwick Ave
Number of Docks: 22
Available Docks: 20
Last Communicat

Finally, let's clean up and close our database connection.

In [13]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
engine.execute(drop_table_query)